In [1]:
from os import listdir
from os.path import isfile, join
mypath = 'txt/'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [2]:
names = open('names.txt', 'w')
for name in onlyfiles:
    names.write(name + '\n')
names.close()

In [3]:
from collections import defaultdict

ind = defaultdict()

for i in range(len(onlyfiles)):
    ind[onlyfiles[i]] = i
    
files = []
for path in onlyfiles:
    file = open(mypath + path, 'r').read()
    files.append(file)
    
words = []
for file in files:
    words.append(file.split(' '))
    
all_words = []
for w in words:
    all_words.extend(w)
all_words_set = list(set(all_words))

ind_word = defaultdict()
for i in range(len(all_words_set)):
    ind_word[all_words_set[i]] = i

In [4]:
import numpy as np
A = np.zeros((len(all_words_set), len(files)))
for i in range(len(words)):
    for j in range(len(words[i])):
        A[ind_word[words[i][j]], i] += 1
        
full_counts = np.sum(A, axis=1)
P = (A.T / full_counts).T

P[P==0] = 1e-100

log_P = np.log(P)
log_P = log_P / np.log(len(words))
g = np.sum(P * log_P, axis = 1)
A_norm = (np.log(A+1).T * g).T

In [5]:
from scipy.linalg import svd

T, s, D = svd(A_norm, full_matrices=False)

In [6]:
r = 300
T_my = T[:, :r]
s_my = s[:r]

In [7]:
s_my = 1 / s_my

In [8]:
trans = T_my.dot(np.diag(s_my))

In [9]:
vects = A_norm.T.dot(trans)

In [10]:
result = np.zeros((len(words), len(words)))

In [11]:
for i in range(len(words)):
    for j in range(len(words)):
        result[i, j] = np.sum(vects[i,:] * vects[j,:]) / \
        np.sqrt(np.sum(vects[i,:]* vects[i,:]) * np.sum(vects[j,:] * vects[j,:]))

In [12]:
file = open('table.tsv', 'w')
for i in range(len(words)):
    for j in range(len(words)):
        if j != len(words)-1:
            file.write(str(result[i, j]) + '\t')
        else:
            file.write(str(result[i, j]))
    file.write('\n')
file.close()

In [13]:
import pandas as pd
matr = pd.read_csv('table.tsv', header=None, sep='\t')

In [15]:
top_ind = (result * -1).argsort(axis = 1)

In [16]:
tops = open('tops.txt', 'w')
for i in range(len(onlyfiles)):
    tops.write('file number: ' + onlyfiles[i][:4] + '; ')
    tops.write('top-5 numbers: ')
    for j in top_ind[i,1:6]:
        tops.write(onlyfiles[j][:4] + ' ')
    tops.write('\n')
tops.close()